In [1]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re


from nltk.corpus import gutenberg, stopwords
from collections import Counter

from sklearn import linear_model,neighbors, ensemble, svm
from sklearn.model_selection import train_test_split

%matplotlib inline

/root/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
            
        
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df
    


In [3]:
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [4]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [5]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [6]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [7]:
#Load Results of bow_features

emma_bow = pd.read_csv('emma_bow.csv')
moby_bow = pd.read_csv('moby_bow.csv')
word_counts = pd.read_csv('aliceper_wcs.csv')
word_counts.drop(['Unnamed: 0'],1,inplace=True)
moby_bow.drop(['Unnamed: 0'],1,inplace=True)
emma_bow.drop(['Unnamed: 0'],1,inplace=True)

In [8]:
word_counts.head()

,tea,offensive,attendance,pursue,devotion,dear,resent,parent,distant,furiously,...,clamour,somewhat,tread,sunday,laughing,wound,astonish,maintain,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Alice was beginning to get very tired of sitti...,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,So she was considering in her own mind (as wel...,Carroll
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,There was nothing so VERY remarkable in that; ...,Carroll
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,Oh dear!,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,I shall be late!',Carroll


In [9]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [10]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.9579937304075236

Test set score: 0.9158834586466166


/root/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Challange 0. Add features and score better than 90% avg on cross val

In [11]:
#add sentence length
word_counts['sent_len'] = [len(i) for i in word_counts.text_sentence]

In [12]:
#Loop to count posses in each sentence
asents = list(alice_doc.sents)
psents = list(persuasion_doc.sents)
both_sents = asents+psents
posses = []
posito = []
for i,sentence in enumerate(both_sents):    
    pss = []
    for token in sentence:
        pss.append(token.pos_)
        posses.append(token.pos_)
    posito.append(pss)

In [13]:
#Create df for posses
posses_df = pd.DataFrame(columns=set(posses))
posses_df['poss'] = posito
posses_df.loc[:, posses] = 0



In [14]:
posses_df.head(2)

,PUNCT,NOUN,CCONJ,NUM,PROPN,INTJ,PRON,X,PART,DET,ADJ,VERB,ADP,ADV,poss
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[PROPN, VERB, VERB, PART, VERB, ADV, ADJ, ADP,..."
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[ADV, PRON, VERB, VERB, ADP, ADJ, ADJ, NOUN, P..."


In [15]:
#turn poss counts into features
for i, sentence in enumerate(posses_df['poss']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = posses_df.poss[i]
        
        # Populate the row with word counts.
        for word in words:
            posses_df.loc[i, word] += 1

In [16]:
posses_df.head()

,PUNCT,NOUN,CCONJ,NUM,PROPN,INTJ,PRON,X,PART,DET,ADJ,VERB,ADP,ADV,poss
0,10,12,6,0,2,0,3,0,2,5,3,13,8,3,"[PROPN, VERB, VERB, PART, VERB, ADV, ADJ, ADP,..."
1,7,8,2,0,2,0,4,0,1,6,7,11,8,7,"[ADV, PRON, VERB, VERB, ADP, ADJ, ADJ, NOUN, P..."
2,4,2,1,0,2,2,2,0,1,3,1,5,4,6,"[ADV, VERB, NOUN, ADV, ADV, ADJ, ADP, DET, PUN..."
3,1,0,0,0,0,2,0,0,0,0,0,0,0,0,"[INTJ, INTJ, PUNCT]"
4,2,0,0,0,0,0,1,0,0,0,1,2,0,0,"[PRON, VERB, VERB, ADJ, PUNCT, PUNCT]"


In [17]:
dfv2 = pd.concat([word_counts,posses_df.drop(['poss'],1).reset_index(drop=True)],axis=1)
dfv2.head(2)

,tea,offensive,attendance,pursue,devotion,dear,resent,parent,distant,furiously,...,PROPN,INTJ,PRON,X,PART,DET,ADJ,VERB,ADP,ADV
0,0,0,0,0,0,0,0,0,0,0,...,2,0,3,0,2,5,3,13,8,3
1,0,0,0,0,0,0,0,0,0,0,...,2,0,4,0,1,6,7,11,8,7


In [18]:
dfv2.isnull().sum().sum()

0

...(Excccelllennt)[mr. burns twiddles fingers]

In [19]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

In [20]:
#try SVC with scaling the X test
Y = dfv2['text_source']
X = np.array(dfv2.drop(['text_sentence','text_source'], 1))
X = scale.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

/root/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/root/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [21]:
svc = svm.SVC(kernel='linear', C=.9)

svc.fit(X_train,y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))


Training set score: 0.9918495297805643

Test set score: 0.8641917293233082


In [22]:
#try without scaling
Y = dfv2['text_source']
X = np.array(dfv2.drop(['text_sentence','text_source'], 1))
#X = scale.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [23]:
svc = svm.SVC(kernel='linear', C=.9)

svc.fit(X_train,y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.9771159874608151

Test set score: 0.9130639097744361


In [24]:
#better without scaling, tweak C paramenter
svc = svm.SVC(kernel='linear', C=.8)

svc.fit(X_train,y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.9752351097178683

Test set score: 0.9154135338345865


In [25]:
#better results as C goes down, the model score decreasing well on the training set, 
#but increasingly well on the test set
#tweak C paramenter more
svc = svm.SVC(kernel='linear', C=.7)

svc.fit(X_train,y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.9717868338557993

Test set score: 0.9149436090225563


In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(svc,X,Y,cv=5)

In [26]:
svc = svm.SVC(kernel='linear', C=.35)

svc.fit(X_train,y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.9570532915360501

Test set score: 0.9163533834586466


,feature,coef
747,alice,1.826308
1888,be,1.493794
1581,will,1.420623
2276,not,1.346527
1682,would,1.149408


In [ ]:
pos_coef_feats = feature_sigs[feature_sigs['coef'] > 0]['feature']
dfv4 = dfv4[pos_coef_feats]

In [ ]:
cross_val_score(svc,X,Y,cv=5)

In [27]:
#try lr1
lrc = LogisticRegression(C=.94,penalty='l2')
lrc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lrc.score(X_train, y_train))
print('\nTest set score:', lrc.score(X_test, y_test))

/root/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(3190, 3077) (3190,)
Training set score: 0.9589341692789969

Test set score: 0.918703007518797


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

In [ ]:
X_pca = pd.DataFrame(pca.fit_transform(X))

In [ ]:
pca_df = pd.concat([X_pca,Y],axis=1)

In [ ]:
sns.scatterplot(pca_df[1],pca_df[0],hue='text_source',data=pca_df)

What a messy plot, lets start semi over

In [29]:
posses_df['text_source']= sentences[1]
posses_df.head(2)

,PUNCT,NOUN,CCONJ,NUM,PROPN,INTJ,PRON,X,PART,DET,ADJ,VERB,ADP,ADV,poss,text_source
0,10,12,6,0,2,0,3,0,2,5,3,13,8,3,"[PROPN, VERB, VERB, PART, VERB, ADV, ADJ, ADP,...",Carroll
1,7,8,2,0,2,0,4,0,1,6,7,11,8,7,"[ADV, PRON, VERB, VERB, ADP, ADJ, ADJ, NOUN, P...",Carroll


In [30]:
posses_df['sent_len'] = [len(i) for i in word_counts.text_sentence]
posses_df.head()

,PUNCT,NOUN,CCONJ,NUM,PROPN,INTJ,PRON,X,PART,DET,ADJ,VERB,ADP,ADV,poss,text_source,sent_len
0,10,12,6,0,2,0,3,0,2,5,3,13,8,3,"[PROPN, VERB, VERB, PART, VERB, ADV, ADJ, ADP,...",Carroll,301
1,7,8,2,0,2,0,4,0,1,6,7,11,8,7,"[ADV, PRON, VERB, VERB, ADP, ADJ, ADJ, NOUN, P...",Carroll,289
2,4,2,1,0,2,2,2,0,1,3,1,5,4,6,"[ADV, VERB, NOUN, ADV, ADV, ADJ, ADP, DET, PUN...",Carroll,140
3,1,0,0,0,0,2,0,0,0,0,0,0,0,0,"[INTJ, INTJ, PUNCT]",Carroll,8
4,2,0,0,0,0,0,1,0,0,0,1,2,0,0,"[PRON, VERB, VERB, ADJ, PUNCT, PUNCT]",Carroll,17


In [31]:
X = posses_df.drop(['poss','text_source'],1)
y = posses_df.text_source

In [32]:
X = posses_df.drop(['poss','text_source'],1)
y = posses_df.text_source

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.4,random_state=43)


In [34]:
#try lr1
lrc = LogisticRegression(C=.94,penalty='l2')
lrc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lrc.score(X_train, y_train))
print('\nTest set score:', lrc.score(X_test, y_test))


(3190, 15) (3190,)
Training set score: 0.761128526645768

Test set score: 0.7504699248120301


/root/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [86]:
# lets add people??
peoplea = [entity.text for entity in list(alice_doc.ents) if entity.label_ == "PERSON"]
peoplep = [entity.text for entity in list(persuasion_doc.ents) if entity.label_ == "PERSON"]


In [87]:
#combine the sets of 'people'
persons = set(peoplea).union(set(peoplep))

In [88]:
#Create df for posses
people = pd.DataFrame(columns=persons)
people['text_sentence'] = sentences[0]
people['text_source']= sentences[1]
people.loc[:, persons] = 0

In [89]:
people.head()

,Duchess,Soup,Elliot,Mrs Charles,Frederick,Lady Russell,Repeat,Basil,Sophia,Robinson,...,Mrs,WILLIAM,the King,Grappler,William Walter Elliot,William,the Mock Turtle,Elsie,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [128]:
#turn people counts into features
for i, sentence in enumerate(people['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = people['text_sentence'][i]
        # Populate the row with word counts.
        for word in words:
            if str(word) in persons:           
                people.loc[i, str(word)] += 1

In [133]:
peeps = people.drop(['text_sentence','text_source'],1).reset_index(drop=True)

In [169]:
#make new model
dfv3 = pd.concat([posses_df.drop(['poss','text_source'],1).reset_index(drop=True),
                  people.drop(['text_sentence'],1
                             ).reset_index(drop=True)],axis=1)
dfv4 = pd.concat([word_counts.drop(['text_sentence'],1),
                  posses_df.drop(['poss','text_source'],1).reset_index(drop=True),
                  people.drop(['text_sentence','text_source'],1
                             ).reset_index(drop=True)],axis=1)

In [170]:
dfv3.head(2)

,PUNCT,NOUN,CCONJ,NUM,PROPN,INTJ,PRON,X,PART,DET,...,Hush,Mrs,WILLIAM,the King,Grappler,William Walter Elliot,William,the Mock Turtle,Elsie,text_source
0,10,12,6,0,2,0,3,0,2,5,...,0,0,0,0,0,0,0,0,0,Carroll
1,7,8,2,0,2,0,4,0,1,6,...,0,0,0,0,0,0,0,0,0,Carroll


In [171]:
dfv4.head(2)

,tea,offensive,attendance,pursue,devotion,dear,resent,parent,distant,furiously,...,Mary Ann,Hush,Mrs,WILLIAM,the King,Grappler,William Walter Elliot,William,the Mock Turtle,Elsie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
#just people and pos
X = dfv3.drop(['text_source'],1)
y = dfv3.text_source

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.4,random_state=43)


In [163]:
#try lr1
lrc = LogisticRegression(C=1500,penalty='l2')
lrc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lrc.score(X_train, y_train))
print('\nTest set score:', lrc.score(X_test, y_test))


(3190, 273) (3190,)
Training set score: 0.841692789968652

Test set score: 0.8477443609022557


/root/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [177]:
#all together
X = dfv4.drop(['text_source'],1)
y = dfv4.text_source

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.4,random_state=43)

lrc = LogisticRegression(C=150,penalty='l2')
lrc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lrc.score(X_train, y_train))
print('\nTest set score:', lrc.score(X_test, y_test))

/root/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(3190, 3336) (3190,)
Training set score: 0.9927899686520376

Test set score: 0.9017857142857143


In [209]:
svc = svm.SVC(kernel='linear', C=.35)

svc.fit(X_train,y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.9639498432601881

Test set score: 0.9135338345864662


In [214]:
svc = svm.SVC(kernel='linear', C=.3)

svc.fit(X_train,y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.9608150470219435

Test set score: 0.9144736842105263


In [ ]:
word_counts.to_csv('aliceper_wcs.csv')
emma_bow.to_csv('emma_bow.csv')
moby_bow.to_csv('moby_bow.csv')